In [93]:
import pandas as pd
import matplotlib.pyplot as plt

In [94]:
import spacy

In [95]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

In [96]:
#spacy.prefer_gpu()
#nlp = spacy.load("fr_dep_news_trf")
nlp = spacy.load("fr_core_news_lg")

In [97]:
interventions = pd.read_csv('out/interventions.csv')
acteurs = pd.read_csv('out/interventions.csv')
votes = pd.read_csv('out/votes.csv')
organes = pd.read_csv('out/organes.csv')

In [98]:
interventions

,id_syceron,acteur,code_grammaire,code_style,stime,texte,seance,mots
0,2845790,PA721908,OUV_SEAN_2_1,NORMAL,919.71,La séance est ouverte.,RUANR5L16S2022IDS26244,4
1,2845795,PA721908,ODJ_APPEL_DISCUSSION,NORMAL,922.29,"L’ordre du jour appelle la discussion, sur le ...",RUANR5L16S2022IDS26244,26
2,2845797,PA721908,PAROLE_GENERIQUE,NORMAL,931.96,"La parole est à M. Jean-Renvé Cazeneuve, rap...",RUANR5L16S2022IDS26244,13
3,2845798,PA719472,PAROLE_GENERIQUE,NORMAL,957.22,Je suis ravi de vous présenter les conclusion...,RUANR5L16S2022IDS26244,101
4,2845799,PA642847,INTERRUPTION_1_10,NORMAL,981.52,Un Lorrain !,RUANR5L16S2022IDS26244,3
...,...,...,...,...,...,...,...,...
156971,2944887,PA793382,INTERRUPTION_1_10,NORMAL,9539.76,Et c’est d’acheter du charbon à l’Allemagne ?,RUANR5L16S2023IDS26598,8
156972,2944889,PA720908,SCRUT_PUB_ADT_1_2,NORMAL,9549.08,Je mets aux voix l’amendement no 3039.,RUANR5L16S2023IDS26598,7
156973,2944891,PA720908,SCRUT_PUB_ADT_1_4,NORMAL,9554.61,Voici le résultat du scrutin : Nombre ...,RUANR5L16S2023IDS26598,165
156974,2944893,PA720908,FIN_SEAN_1_0,NORMAL,9555.48,La suite de la discussion est renvoyée à la ...,RUANR5L16S2023IDS26598,11


In [123]:
# https://www.analyticsvidhya.com/blog/2021/06/nlp-application-named-entity-recognition-ner-in-python-with-spacy/

In [114]:
def ner(doc):
    #doc = nlp(text)
    return [ (ent.text, ent.label_) for ent in doc.ents ]

entities = (
    interventions
    #.iloc[0:100]
    .assign(
        #entities = lambda df: df.texte.apply(ner)
        entities = lambda df: [ ner(doc) for doc in nlp.pipe(list(df.texte.astype(str)), n_process=16) ]
    )
)

entities

,id_syceron,acteur,code_grammaire,code_style,stime,texte,seance,mots,entities
0,2845790,PA721908,OUV_SEAN_2_1,NORMAL,919.71,La séance est ouverte.,RUANR5L16S2022IDS26244,4,[]
1,2845795,PA721908,ODJ_APPEL_DISCUSSION,NORMAL,922.29,"L’ordre du jour appelle la discussion, sur le ...",RUANR5L16S2022IDS26244,26,[]
2,2845797,PA721908,PAROLE_GENERIQUE,NORMAL,931.96,"La parole est à M. Jean-Renvé Cazeneuve, rap...",RUANR5L16S2022IDS26244,13,"[(M. Jean-Renvé Cazeneuve, PER)]"
3,2845798,PA719472,PAROLE_GENERIQUE,NORMAL,957.22,Je suis ravi de vous présenter les conclusion...,RUANR5L16S2022IDS26244,101,"[(CMP, MISC), (PLFR, ORG), (Assemblée nationa..."
4,2845799,PA642847,INTERRUPTION_1_10,NORMAL,981.52,Un Lorrain !,RUANR5L16S2022IDS26244,3,"[(Lorrain, LOC)]"
...,...,...,...,...,...,...,...,...,...
156971,2944887,PA793382,INTERRUPTION_1_10,NORMAL,9539.76,Et c’est d’acheter du charbon à l’Allemagne ?,RUANR5L16S2023IDS26598,8,"[(Allemagne, LOC)]"
156972,2944889,PA720908,SCRUT_PUB_ADT_1_2,NORMAL,9549.08,Je mets aux voix l’amendement no 3039.,RUANR5L16S2023IDS26598,7,[]
156973,2944891,PA720908,SCRUT_PUB_ADT_1_4,NORMAL,9554.61,Voici le résultat du scrutin : Nombre ...,RUANR5L16S2023IDS26598,165,[]
156974,2944893,PA720908,FIN_SEAN_1_0,NORMAL,9555.48,La suite de la discussion est renvoyée à la ...,RUANR5L16S2023IDS26598,11,[]


In [118]:
entities_long = (
    entities
    [['id_syceron', 'acteur', 'entities']]
    .explode('entities')
    .dropna(subset="entities")
    .assign(
        entity = lambda df: df.entities.apply(lambda x: x[0]),
        label = lambda df: df.entities.apply(lambda x: x[1]),
    )
)

entities_long

,id_syceron,acteur,entities,entity,label
2,2845797,PA721908,"(M. Jean-Renvé Cazeneuve, PER)",M. Jean-Renvé Cazeneuve,PER
3,2845798,PA719472,"(CMP, MISC)",CMP,MISC
3,2845798,PA719472,"(PLFR, ORG)",PLFR,ORG
3,2845798,PA719472,"(Assemblée nationale, ORG)",Assemblée nationale,ORG
3,2845798,PA719472,"(Sénat, LOC)",Sénat,LOC
...,...,...,...,...,...
156970,2944940,PA794008,"(Russie, LOC)",Russie,LOC
156970,2944940,PA794008,"(Applaudissements, ORG)",Applaudissements,ORG
156970,2944940,PA794008,"(NUPES, ORG)",NUPES,ORG
156971,2944887,PA793382,"(Allemagne, LOC)",Allemagne,LOC


In [142]:
entities_long.to_csv('out/interventions-entities.csv', index=False)

In [141]:
entities_long.query('entity.str.contains("Mélenchon")')

,id_syceron,acteur,entities,entity,label
209,2846112,PA719118,"(Mélenchon !, MISC)",Mélenchon !,MISC
1658,3106851,PA795386,"(Jean-Luc Mélenchon, PER)",Jean-Luc Mélenchon,PER
2747,2894021,PA720468,"(Jean-Luc Mélenchon, PER)",Jean-Luc Mélenchon,PER
2840,2894266,PA331582,"(Jean-Luc Mélenchon, PER)",Jean-Luc Mélenchon,PER
3701,2940924,PA773443,"(M. Mélenchon, PER)",M. Mélenchon,PER
...,...,...,...,...,...
152794,3019865,PA793464,"(De Mélenchon !, MISC)",De Mélenchon !,MISC
153786,2829324,PA718850,"(Jean-Luc Mélenchon, PER)",Jean-Luc Mélenchon,PER
154414,2819674,PA720286,"(M. Mélenchon, MISC)",M. Mélenchon,MISC
154425,2819773,PA720892,"(Jean-Luc Mélenchon, PER)",Jean-Luc Mélenchon,PER


In [126]:
entities_long.query('label == "PER"').entity.value_counts().head(30)

M.                              4792
Mme                             3249
Très                           1777
protéger                       1573
Président de la République    1556
réalité                       1279
État                           1172
impôts                          804
Emmanuel Macron                  798
Madame                           704
Dès                             699
dès                             657
Benjamin Lucas                   433
Mme Emmanuelle Ménard           422
Charles de Courson               391
Mme Sandra Regol                 359
Arthur Delaporte                 345
M. Aurélien Saintoul            338
M. Thibault Bazin                295
M. Bastien Lachaud               289
Marine Le Pen                    288
Macron                           278
M. Ugo Bernalicis                267
M. Antoine Léaument             266
Démocrate                       239
Mme Véronique Louwagie          237
M. Jérôme Guedj                232
M

In [121]:
entities_long.query('label == "LOC"').entity.value_counts().head(30)

la France       5164
France          3515
République     1688
Europe          1613
Sénat          1585
La France       1444
Même           1389
Ukraine         1320
idée            986
Défavorable     962
Paris            836
Mayotte          686
Russie           669
Néanmoins       663
de France        648
État            591
Allemagne        577
TVA              575
enquête         575
Horizons         561
adoptée         540
Hexagone         484
Corse            455
l’Europe         386
États-Unis      367
règles          367
La Réunion      365
Grâce           354
Polynésie       347
Où              326
Name: entity, dtype: int64

In [122]:
entities_long.query('label == "ORG"').entity.value_counts().head(30)

Applaudissements          22108
Gouvernement              12765
LFI                       12521
RE                         9605
RN                         5525
Dem                        4997
NUPES                      4538
État                      4108
SOC                        3841
LR                         3030
Assemblée nationale       2567
GDR                        2379
HOR                        2374
Rassemblement national     2034
élus                      2027
Parlement                  1959
Union européenne          1600
Renaissance                 971
EDF                         944
Constitution                926
Première ministre          852
Socialistes                 727
PLFSS                       722
Conseil                     718
Assemblée                  716
Les Républicains           715
âgées                     587
Conseil d’État             534
Cour des comptes            463
rappelé                    460
Name: entity, dtype: int64

In [127]:
deputes = (
    votes
    .groupby(['acteurRef', 'organe'])
    .agg({'scrutin' : 'count' })
    .reset_index()
    .join(organes.set_index('uid')[['libelleAbrev']], on='organe')
    .set_index('acteurRef')
)

deputes

,organe,scrutin,libelleAbrev
acteurRef,,,
PA1008,PO800496,539,SOC
PA1206,PO793087,119,NI
PA1327,PO800508,309,LR
PA1567,PO800496,410,SOC
PA1592,PO793087,122,NI
...,...,...,...
PA817203,PO800538,358,RE
PA817211,PO800490,386,LFI-NUPES
PA822617,PO793087,21,NI


In [133]:
(
    entities_long
    .join(deputes, on='acteur')
    .query('entity.str.contains("Le Pen")')
    .pivot_table(
        index='libelleAbrev',
        columns='entity',
        values='id_syceron',
        aggfunc='count'
    )
    .sum(axis=1)
    .to_frame()
    # .pipe(lambda df: df.div(df.sum(axis=1), axis=0))
    # .style.background_gradient(axis=1)
)

,0
libelleAbrev,
DEM,23.0
ECOLO,42.0
GDR-NUPES,9.0
HOR,18.0
LFI-NUPES,90.0
LIOT,9.0
LR,22.0
NI,17.0
RE,132.0


In [136]:
(
    entities_long
    .join(deputes, on='acteur')
    .query('entity.str.contains("Macron")')
    .pivot_table(
        index='libelleAbrev',
        columns='entity',
        values='id_syceron',
        aggfunc='count'
    )
    .sum(axis=1)
    .to_frame()
 )

,0
libelleAbrev,
DEM,11.0
ECOLO,111.0
GDR-NUPES,89.0
HOR,6.0
LFI-NUPES,625.0
LIOT,24.0
LR,123.0
NI,81.0
RE,60.0
